In [1]:
!pip install datasets transformers peft accelerate evaluate nltk onnx onnxruntime gradio optimum[onnxruntime]

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

'1'

In [3]:
## Cell 2 — Imports
import torch
from pathlib import Path
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
    pipeline,
)
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [4]:
## Cell 3 — Load dataset
dataset = load_dataset("halaction/song-lyrics", split="train[:1000]")
print(dataset.column_names)
print(dataset[0])

['lyrics', 'genre']
{'lyrics': "[Intro: Method Man w/ sample] + (Sunny valentine). We got butter (8X). (The gun'll go the gun'll go.... The gun'll go...). [Raekwon]. Aiyo one thing for sure keep you of all. Keep a nice crib fly away keep to the point. Keep niggaz outta ya face who snakes. Keep bitches in they place keep the mac in a special place. Keep moving for papes keep cool keep doing what you doing. Keep it fly keep me in the crates. Cuz I will erase shit on the real note you'se a waste. It's right here for you I will lace you. Rip you and brace you put a nice W up on ya face. Word to mother you could get chased. It's nothing to taste blood on a thug if he gotta go. All I know is we be giving grace. This is a place from where we make tapes. We make 'em everywhere still in all we be making base. Y'all be making paste these little niggaz they be making shapes. Our shit is art yours is traced. [Chorus: Sunny Valentine]. This is the way that we rolling in the streets. You know when w

In [5]:
## Cell 4 — Choose text column + clean rows
candidate_columns = ["lyrics", "text", "song", "content"]
text_col = next((c for c in candidate_columns if c in dataset.column_names), dataset.column_names[0])

dataset = dataset.filter(lambda x: x[text_col] is not None and x[text_col].strip() != "")
dataset = dataset.select_columns([text_col])
print("Using text column:", text_col)
print("Rows:", len(dataset))

Using text column: lyrics
Rows: 1000


In [6]:
## Cell 5 — Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 pad token fix
max_length = 128

In [7]:
## Cell 6 — Tokenization function + map

def tokenize_function(batch):
    texts = [t + tokenizer.eos_token for t in batch[text_col]]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # Causal LM labels
    return tokenized

tokenized_data = dataset.map(tokenize_function, batched=True, remove_columns=[text_col])
tokenized_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

splits = tokenized_data.train_test_split(test_size=0.1, seed=42)
train_dataset = splits["train"]
eval_dataset = splits["test"]

print(train_dataset[0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [8]:
## Cell 7 — Load GPT-2 and apply LoRA

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],
    bias="none",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\peft\tuners\lora\layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [9]:
## Cell 8 — Training arguments (includes weight decay)

common_args = dict(
    output_dir="./gpt2-lyrics-lora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=5,
    weight_decay=0.01,  # required regularization
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=20,
    fp16=torch.cuda.is_available(),
    report_to="none",
)

# transformers version compatibility:
# - older versions use evaluation_strategy
# - newer versions use eval_strategy
try:
    training_args = TrainingArguments(evaluation_strategy="epoch", **common_args)
except TypeError:
    training_args = TrainingArguments(eval_strategy="epoch", **common_args)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [10]:
## Cell 9 — Trainer with early stopping

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

C:\Users\shweiss\AppData\Local\Temp\ipykernel_20024\2783257490.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
## Cell 10 — Train + evaluate

trainer.train()
metrics = trainer.evaluate()
print(metrics)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.683700,3.362211
2,3.596300,3.334249
3,3.545200,3.319573
4,3.542200,3.313650
5,3.529300,3.312150


C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
C:\Users\shweiss\AppData\Local\a

{'eval_loss': 3.3121497631073, 'eval_runtime': 45.5956, 'eval_samples_per_second': 2.193, 'eval_steps_per_second': 0.548, 'epoch': 5.0}


In [12]:
## Cell 11 — Save LoRA adapter + merged model

trainer.model.save_pretrained("./gpt2-lyrics-lora-adapter")
tokenizer.save_pretrained("./gpt2-lyrics-lora-adapter")

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./gpt2-lyrics-merged")
tokenizer.save_pretrained("./gpt2-lyrics-merged")

('./gpt2-lyrics-merged\\tokenizer_config.json',
 './gpt2-lyrics-merged\\special_tokens_map.json',
 './gpt2-lyrics-merged\\vocab.json',
 './gpt2-lyrics-merged\\merges.txt',
 './gpt2-lyrics-merged\\added_tokens.json')

In [13]:
## Cell 12 — Export to ONNX

from optimum.onnxruntime import ORTModelForCausalLM

onnx_dir = Path("./gpt2-lyrics-onnx")
onnx_dir.mkdir(parents=True, exist_ok=True)

ort_model = ORTModelForCausalLM.from_pretrained("./gpt2-lyrics-merged", export=True)
ort_model.save_pretrained(onnx_dir)
tokenizer.save_pretrained(onnx_dir)

print("ONNX export complete:", onnx_dir.resolve())

`torch_dtype` is deprecated! Use `dtype` instead!
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\optimum\exporters\base.py:151: FutureWarning: functools.partial will be a method descriptor in future Python versions; wrap it in staticmethod() if you want to preserve the old behavior
  self._normalized_config = self.NORMALIZED_CONFIG_CLASS(self._config)
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\transformers\cache_utils.py:132: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if not self.is_initialized or self.keys.numel() == 0:
C:\Users\shweiss\AppData\Local\anaconda3\Lib\site-packages\transformers\masking_utils.py:207: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of 

ONNX export complete: C:\Users\shweiss\gpt2-lyrics-onnx


In [14]:
## Cell 13 — Generate text helper

device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model="./gpt2-lyrics-merged", tokenizer=tokenizer, device=device)

def generate_lyrics(prompt, max_new_tokens=80, temperature=0.9, top_p=0.95):
    output = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=1.1,
        num_return_sequences=1,
    )[0]["generated_text"]
    return output

Device set to use cpu


In [15]:
## Cell 14 — Gradio app

import gradio as gr

demo = gr.Interface(
    fn=generate_lyrics,
    inputs=[
        gr.Textbox(lines=2, label="Prompt"),
        gr.Slider(20, 200, value=80, step=1, label="max_new_tokens"),
        gr.Slider(0.1, 1.5, value=0.9, step=0.05, label="temperature"),
        gr.Slider(0.5, 1.0, value=0.95, step=0.01, label="top_p"),
    ],
    outputs=gr.Textbox(lines=8, label="Generated Lyrics"),
    title="GPT-2 Lyrics Generator (LoRA Fine-Tuned)",
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [16]:
## Cell 15 — BLEU evaluation + qualitative review notes

prompt = "I walk alone beneath the city lights"
generated_text = generate_lyrics(prompt, max_new_tokens=50)
print("Generated Lyrics:\n", generated_text)

# Replace this with a real reference continuation from a held-out lyric sample
reference_text = "I walk alone beneath the city lights, chasing echoes of a fading night"
reference = [reference_text.split()]
candidate = generated_text.split()

bleu_score = sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1)
print("BLEU Score:", bleu_score)

print("\nQualitative checklist:")
print("- Coherence: Does the text stay on a consistent theme?")
print("- Relevance: Does it continue the prompt naturally?")
print("- Creativity: Are wording and imagery varied?")
print("- Fluency: Does it read smoothly?")

Generated Lyrics:
 I walk alone beneath the city lights. Like a star gazing sky overhead to keep me awake at night when I need something else on my side... The air feels like heaven, as if there's nothing left of it but tears and hope for more.. And every single inch in this world
BLEU Score: 0.09548024812323944

Qualitative checklist:
- Coherence: Does the text stay on a consistent theme?
- Relevance: Does it continue the prompt naturally?
- Creativity: Are wording and imagery varied?
- Fluency: Does it read smoothly?
